## Read short story for tokenzation

### Step 1: Create tokens

In [60]:
with open("the-verdict.txt", 'r', encoding='utf-8') as f:
    raw_text = f.read()

# Print total number of characters
print("Total number of characters:", len(raw_text))

# print first 100 characters
print(raw_text[:99])


Total number of characters: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [61]:
# Use regular expression library
import re

text = "Hello world! This is a test, working now."

# Split based on white spaces: use \s as regex
result = re.split(r'\s', text) # This will remove the white space

print(result)

result = re.split(r'(\s)', text) # This will include the white spaces
print(result)


['Hello', 'world!', 'This', 'is', 'a', 'test,', 'working', 'now.']
['Hello', ' ', 'world!', ' ', 'This', ' ', 'is', ' ', 'a', ' ', 'test,', ' ', 'working', ' ', 'now.']


In [62]:
# Now we want to include punctuation characters (commas full stops and exclamation marks) also as separate strings
result = re.split(r'(\W)', text)
print(result)

['Hello', ' ', 'world', '!', '', ' ', 'This', ' ', 'is', ' ', 'a', ' ', 'test', ',', '', ' ', 'working', ' ', 'now', '.', '']


In [63]:
# Remove empty strings and white space characters now
result = [item for item in result if item.strip()]

print(result)

['Hello', 'world', '!', 'This', 'is', 'a', 'test', ',', 'working', 'now', '.']


In [64]:
# for the book text, there are ? -- : _ ; ' " () are also present. Hence we need to consider them while splitting
# Hence our tokenizer code is as follows, as of now. This is a simple tokenizer. 
# For LLMs, different tokenizer scheme is used which we will see later
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [t for t in preprocessed if t.strip()]
print (len(preprocessed))

4690


### Step 2: Creating token IDs

In [65]:
# Get all unique words and sorted
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print(vocab_size)

1130


In [66]:
# Create a vocabulary
vocab = {}
counter = 0
for word in all_words:
    vocab[word] = counter
    counter = counter + 1

items = list(vocab.items())[:20]
for key,value in items:
    print(f"({key}, {value})")

(!, 0)
(", 1)
(', 2)
((, 3)
(), 4)
(,, 5)
(--, 6)
(., 7)
(:, 8)
(;, 9)
(?, 10)
(A, 11)
(Ah, 12)
(Among, 13)
(And, 14)
(Are, 15)
(Arrt, 16)
(As, 17)
(At, 18)
(Be, 19)


In [67]:
# Now we build a simple tokenizer class that will be used to encode the given text to feed IDs to LLM and decode the Token IDs from LLM to convert back to text.
class SimpleTokenizerV1:
    def __init__(self, vocab):
        # For encoder
        self.str_to_int = vocab

        # For decoder, int to string mapping
        self.int_to_str = {i:s for s, i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])

        # Get rid of spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [68]:
print(vocab['"'])

1


In [69]:
# initialize the tokenizer
tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know,"
           Mrs. Gisburn said with pardonable pride."""

ids = tokenizer.encode(text)
print(ids)


[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [70]:
decoded_text = tokenizer.decode(ids)
print(decoded_text)

" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


In [71]:
# The encoder fails when the words are not in the vocabulary
text = "Hello, do you like tea?"
print(tokenizer.encode(text))

KeyError: 'Hello'

#### This shows that we need a large and diverse training set to extend the vocabulary when working on LLMs.

Another solution is to use special context tokens, user by GPT as well.

### SPECIAL CONTEXT TOKENS <|unk|> and <|endoftext|> to existing vocabulary

In [72]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token:integer for integer, token in enumerate(all_tokens)}
len(vocab.items())

1132

In [73]:
# print last 5 entries in the updated vocab
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [74]:
# Updated existing tokenizer class to include the new special tokens
class SimpleTokenizerV2:
    def __init__(self, vocab):
        # For encoder
        self.str_to_int = vocab

        # For decoder, int to string mapping
        self.int_to_str = {i:s for s, i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])

        # Get rid of spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [75]:
tokenizer = SimpleTokenizerV2(vocab)
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))
print(text)


Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [76]:
re.split(r'([,.:;?_!"()\']|--|\s)', text)

['Hello',
 ',',
 '',
 ' ',
 'do',
 ' ',
 'you',
 ' ',
 'like',
 ' ',
 'tea',
 '?',
 '',
 ' ',
 '<|endoftext|>',
 ' ',
 'In',
 ' ',
 'the',
 ' ',
 'sunlit',
 ' ',
 'terraces',
 ' ',
 'of',
 ' ',
 'the',
 ' ',
 'palace',
 '.',
 '']

In [77]:
text

'Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.'

In [78]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [79]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'